In [1]:
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)   
# pd.set_option('display.max_colwidth', None) 


import snowflake.connector
import os
import pandas as pd

# ENTER YOUR SNOWFLAKE CREDENTIALS
# If you don't have them here, or as environment variables, you'll be prompted on run.
SNOWFLAKE_USER = "" 
SNOWFLAKE_PASSWORD = ""

if SNOWFLAKE_USER == "":
    SNOWFLAKE_USER = os.environ.get("SNOWFLAKE_USER")
    if not SNOWFLAKE_USER or SNOWFLAKE_USER == "":
        SNOWFLAKE_USER = input("Enter your Snowflake username: ")
if SNOWFLAKE_PASSWORD == "":
    SNOWFLAKE_PASSWORD = os.environ.get("SNOWFLAKE_PASSWORD")
    if not SNOWFLAKE_PASSWORD or SNOWFLAKE_PASSWORD == "":
        SNOWFLAKE_PASSWORD = input("Enter your Snowflake password: ")

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account='fougxvx-fb30115',
    warehouse='BOTANALYTICS_WH',
)

# Create a cursor object
cur = conn.cursor()



In [4]:
from datetime import timedelta
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)   
pd.set_option('display.max_colwidth', None) 


# df = pd.read_csv('testerResultsNBF.csv')
 # where site_id in ('walmart-tx-newbraunfels-site1') 
cur.execute(f"""
            select BOT_ID, site_id,DATEADD(HOUR,-5,DATETIME) as datetime, storeason, hierarchy_component, event, subevent, actionecode, swversion, location_x, location_y,location_level, filename, velocity_linear, actions_0,transfertype
            from SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
           
            where site_id in ('walmart-tx-newbraunfels-site1') 
           
                AND event = 'collisionDetected'
                AND location_level != 0
                AND DATEADD(HOUR, -5, DATETIME) >= '2025-07-28 00:00:00' 
                AND DATEADD(HOUR, -5, DATETIME) <= '2025-07-29 00:00:00'
            order by datetime
        """)

rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]

df = pd.DataFrame(rows, columns=cols)

high_mag = df[df['SUBEVENT'] == 'highMagnitudeCollision'].copy()


print(type(df['LOCATION_Y']))
df['LOCATION_Y'] = pd.to_numeric(df['LOCATION_Y'], errors='coerce')
df['LOCATION_X'] = pd.to_numeric(df['LOCATION_X'], errors='coerce')

# df = df[(df['LOCATION_Y'] >= 0) & (df['LOCATION_Y'] <= 8)]
df['DATETIME'] = pd.to_datetime(df['DATETIME'], errors='coerce')
df['DAY'] = df['DATETIME'].dt.strftime('%B %#d') 
# df['LOCATION_LEVEL'] = 7

df = df.sort_values(by='DATETIME')

df['GROUP_ID'] = np.nan  

def is_close(instance1, instance2):
    if(instance1['SUBEVENT'] == 'highMagnitudeCollision' and instance2['SUBEVENT'] == 'highMagnitudeCollision'):
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=300) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 5.0 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 5.0 and
        instance1['BOT_ID'] != instance2['BOT_ID']
    )
    elif(instance1['SUBEVENT'] == 'highMagnitudeCollision' or instance2['SUBEVENT'] == 'highMagnitudeCollision'):
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=120) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 2.5 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 2.5 and
        instance1['BOT_ID'] != instance2['BOT_ID']
    )
    else:
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=60) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 2.5 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 2.5 and
        instance1['BOT_ID'] != instance2['BOT_ID']
        )

    

group_id = 1

for day in df['DAY'].unique():
    for level in df['LOCATION_LEVEL'].dropna().unique():
        dfFiltered = df[(df['DAY'] == day) & (df['LOCATION_LEVEL'] == level)].copy()
        dfFiltered.reset_index(inplace=True)

        seen = set()

        for idx, row in dfFiltered.iterrows():
            if idx in seen:
                continue

            group = [idx]
            queue = [idx]
            seen.add(idx)

            while queue:
                current_idx = queue.pop()
                current_row = dfFiltered.loc[current_idx]

                for other_idx, other_row in dfFiltered.iterrows():
                    if other_idx in seen:
                        continue
                    if is_close(current_row, other_row):
                        group.append(other_idx)
                        queue.append(other_idx)
                        seen.add(other_idx)
 
            if len(set(dfFiltered.loc[group, 'BOT_ID'])) > 1:
                df.loc[dfFiltered.loc[group, 'index'], 'GROUP_ID'] = group_id
                group_id += 1
        
high_mag = df[df['SUBEVENT'] == 'highMagnitudeCollision'].copy()


result_df = df[df['GROUP_ID'].notna()].copy()

high_mag['BOT_ID'] = high_mag['BOT_ID'].astype(str).str.strip()
result_df['BOT_ID'] = result_df['BOT_ID'].astype(str).str.strip()


merged = high_mag.merge(
    result_df[['BOT_ID', 'DATETIME']].drop_duplicates(),
    on=['BOT_ID', 'DATETIME'],
    how='left',
    indicator=True
)

for i in range(1, group_id + 1):
    dfGroup = result_df[result_df['GROUP_ID'] == i]
    if 'highMagnitudeCollision' not in dfGroup['SUBEVENT'].values:
            result_df = result_df[result_df['GROUP_ID'] != i]


result_df = result_df.sort_values(
    by=['GROUP_ID', 'SUBEVENT'],
    key=lambda col: (
        col if col.name != 'SUBEVENT' else col != 'highMagnitudeCollision'  
    )
)
solo_high_mag = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge']).reset_index(drop=True)
result_df = result_df.drop_duplicates(subset=['BOT_ID', 'GROUP_ID'])




## groupings
display(result_df)

## solo high mags
display(solo_high_mag)

<class 'pandas.core.series.Series'>


,BOT_ID,SITE_ID,DATETIME,STOREASON,HIERARCHY_COMPONENT,EVENT,SUBEVENT,ACTIONECODE,SWVERSION,LOCATION_X,LOCATION_Y,LOCATION_LEVEL,FILENAME,VELOCITY_LINEAR,ACTIONS_0,TRANSFERTYPE,DAY,GROUP_ID
2443,30848,walmart-tx-newbraunfels-site1,2025-07-28 08:52:59.951,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-46.090309,1.175315,6,bot_30848_20250728_135346_UTC.ion,0.9963366985,1160557,None,July 28,43.0
2460,31580,walmart-tx-newbraunfels-site1,2025-07-28 08:53:48.930,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-48.004234,1.683701,6,bot_31580_20250728_135528_UTC.ion,-1.4133974314,1170417,None,July 28,43.0
202,8502,walmart-tx-newbraunfels-site1,2025-07-28 03:41:08.508,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-9.670397,2.321478,1,bot_8502_20250728_084132_UTC.ion,4.0236625671,1156395,None,July 28,57.0
203,23923,walmart-tx-newbraunfels-site1,2025-07-28 03:41:08.919,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-7.925619,1.380305,1,bot_23923_20250728_084131_UTC.ion,2.6532981396,1025976,None,July 28,57.0
204,8218,walmart-tx-newbraunfels-site1,2025-07-28 03:41:13.082,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-10.447545,3.259885,1,bot_8218_20250728_084133_UTC.ion,1.6093045473,1033477,None,July 28,57.0
803,24224,walmart-tx-newbraunfels-site1,2025-07-28 05:31:28.688,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,24.896999,9.934909,1,bot_24224_20250728_103155_UTC.ion,0.3268309236,1161149,None,July 28,61.0
805,8440,walmart-tx-newbraunfels-site1,2025-07-28 05:31:28.744,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,23.701845,6.262950,1,bot_8440_20250728_103150_UTC.ion,2.3096153736,1125870,None,July 28,61.0
835,24586,walmart-tx-newbraunfels-site1,2025-07-28 05:35:04.453,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,26.226662,6.450326,1,bot_24586_20250728_103520_UTC.ion,-0.4411543608,1157946,None,July 28,61.0
3849,10614,walmart-tx-newbraunfels-site1,2025-07-28 12:19:36.165,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,22.425255,1.117376,1,bot_10614_20250728_171956_UTC.ion,0.0442547984,1008713,None,July 28,69.0
3851,8243,walmart-tx-newbraunfels-site1,2025-07-28 12:19:36.280,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,22.782505,2.331219,1,bot_8243_20250728_171957_UTC.ion,3.5338642597,1069068,None,July 28,69.0


,BOT_ID,SITE_ID,DATETIME,STOREASON,HIERARCHY_COMPONENT,EVENT,SUBEVENT,ACTIONECODE,SWVERSION,LOCATION_X,LOCATION_Y,LOCATION_LEVEL,FILENAME,VELOCITY_LINEAR,ACTIONS_0,TRANSFERTYPE,DAY,GROUP_ID
0,22422,walmart-tx-newbraunfels-site1,2025-07-28 02:54:13.541,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,33.212429,6.454889,2,bot_22422_20250728_075433_UTC.ion,0.4546383619,1132598,None,July 28,NaN
1,22369,walmart-tx-newbraunfels-site1,2025-07-28 03:37:50.820,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-92.228806,1.135582,7,bot_22369_20250728_083826_UTC.ion,0.9904770851,1122698,None,July 28,NaN
2,22369,walmart-tx-newbraunfels-site1,2025-07-28 04:32:14.666,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-57.168522,1.139373,7,bot_22369_20250728_093254_UTC.ion,0.9984672666,1122756,None,July 28,NaN
3,8556,walmart-tx-newbraunfels-site1,2025-07-28 04:52:32.990,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-58.385460,2.455573,7,bot_8556_20250728_095252_UTC.ion,0.9457405806,1133058,None,July 28,NaN
4,23994,walmart-tx-newbraunfels-site1,2025-07-28 04:59:31.942,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-41.521393,5.221885,7,bot_23994_20250728_100126_UTC.ion,-0.4488864839,377149,None,July 28,NaN
5,8255,walmart-tx-newbraunfels-site1,2025-07-28 05:20:21.724,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,26.235001,-0.539858,5,bot_8255_20250728_102043_UTC.ion,1.3669290543,1144739,None,July 28,NaN
6,8255,walmart-tx-newbraunfels-site1,2025-07-28 07:32:33.172,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-51.585999,-4.156051,5,bot_8255_20250728_123334_UTC.ion,2.0000770092,1148953,None,July 28,NaN
7,8297,walmart-tx-newbraunfels-site1,2025-07-28 07:33:53.545,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,43.583927,8.454190,2,bot_8297_20250728_123413_UTC.ion,0.9632757902,1100305,None,July 28,NaN
8,22304,walmart-tx-newbraunfels-site1,2025-07-28 07:41:47.333,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,9.993209,0.879070,4,bot_22304_20250728_124226_UTC.ion,0.7461568117,746447,None,July 28,NaN
9,22317,walmart-tx-newbraunfels-site1,2025-07-28 09:06:19.896,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-41.605000,-2.451958,4,bot_22317_20250728_140640_UTC.ion,-1.8293493986,1206722,None,July 28,NaN


In [ ]:
## RUNNING THIS CELL WILL TELL YOU WHAT GROUPING IDS HAVE AT LEAST ONE BOT WITH A STUCK ALARM BUT IT ISN'T NECESSARY

groupKeeper = set()
for _,rows1 in result_df.iterrows():
    if(rows1['GROUP_ID'] not in groupKeeper):
        bot = rows1['BOT_ID']
        start = rows1['DATETIME'] - pd.Timedelta(minutes=5)
        end = rows1['DATETIME'] + pd.Timedelta(minutes=5)
        cur.execute(f"""
            select BOT_ID, site_id,DATEADD(HOUR,-5,DATETIME) as datetime, storeason, hierarchy_component, event, subevent, actionecode, swversion, location_x, location_y,location_level, filename, velocity_linear, actions_0,transfertype

            from SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
            where site_id in ('walmart-tx-newbraunfels-site1') 
                And bot_id = '{bot}'

                AND event = 'alarm'
                AND location_level != 0
                AND DATEADD(HOUR, -5, DATETIME) >= '{start}'
                AND DATEADD(HOUR, -5, DATETIME) <= '{end}'
            order by datetime
        """)

        rows = cur.fetchall()
        cols = [desc[0] for desc in cur.description]

        newDf = pd.DataFrame(rows, columns=cols)
        
        val = False
        for _,newRows in newDf.iterrows():
            if(newRows['SUBEVENT'] == 'stuck' or newRows['SUBEVENT'] == 'estimation_error'):
                val = True
        
        if(val):
            groupKeeper.add(rows1['GROUP_ID'])
    

## RETURNS THE GROUP IDS WITH STUCK ALARMS IN AT LEAST ONE BOT
groupKeeper




